# 📘 Pre-entrega

**Nombre del alumno:**  Capcha Estrada Roberto


### 1️⃣ Carga de datos

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Monto mi drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Verifico que los archivos csv se encuentren en la carpeta datasets
import os
os.listdir('/content/drive/MyDrive/datasets')

In [ ]:
# Defino las rutas de los datasets y los cargo como dataframes
ruta_ventas = '/content/drive/MyDrive/datasets/ventas.csv'
ruta_marketing = '/content/drive/MyDrive/datasets/marketing.csv'
ruta_clientes = '/content/drive/MyDrive/datasets/clientes.csv'

df_ventas = pd.read_csv(ruta_ventas)
df_marketing = pd.read_csv(ruta_marketing)
df_clientes = pd.read_csv(ruta_clientes)

In [ ]:
# Compruebo que se hayan cargado correctamente.
print("\n****** Archivo ventas.csv ******")
print("Shape :",df_ventas.shape)
display(df_ventas.head())
print()
print("_"*80)
print("\n****** Archivo marketing.csv ******")
print("Shape :",df_marketing.shape)
display(df_marketing.head())
print()
print("_"*80)
print("\n****** Archivo clientes.csv ******")
print("Shape :",df_clientes.shape)
display(df_clientes.head())
print("_"*80)


### 2️⃣ Análisis exploratorio inicial

In [ ]:
# Función para realizar un análisis exploratorio inicial de los DataFrames.

def eda(df, nombre):
    print(f"***** {nombre} *****")
    print("_" * 50)
    print()
    print("Info:")
    display(df.info())
    print("_" * 50)
    print()
    print("Shape:", df.shape)
    print("_" * 50)
    print()
    print("Columnas:", list(df.columns))
    print("_" * 50)
    print()
    print("Tipos de datos (dtypes):")
    print(df.dtypes)
    print("_" * 50)
    print()
    print("\nNulos por columna:")
    print(df.isna().sum().to_frame("nulos"))
    print("_" * 50)
    print()
    print("\nPrimeras filas:")
    display(df.head(4))
    print("_" * 50)
    print()
    print("\nUltimas filas:")
    display(df.tail(4))
    print("_" * 50)
    print()
    print("\nDescribe (numérico):")
    display(df.describe(include='number').round(2))
    print("_"*80)

In [ ]:
# *************  EDA VENTAS  **************

eda(df_ventas,"Ventas (EDA)")

### ***Resumen***

Al realizar el análisis exploratorio de **Ventas.csv** se observa:


1.   El dataframe posee 3035 filas y 6 columnas
2.   La columna Precio es object y debería ser flotante (lleva el signo $ delante del número)
1.   La columna Fecha es object y debería ser datetime
4.   Hay cuatro campos nulos (dos en precio y dos en cantidad)





In [ ]:
# EDA MARKETING

eda(df_marketing,"Marketing (EDA)")

### ***Resumen***

Al realizar el análisis exploratorio de **Marketing.csv** se observa:


1.   El dataframe posee 90 filas y 6 columnas
1.   Las columnas fecha_inicio y fecha_fin son object y deberían ser datetime
4.   No hay campos nulos

In [ ]:
# EDA CLIENTES
eda(df_clientes,"Clientes (EDA)")

### ***Resumen***

Al realizar el análisis exploratorio de **Clientes.csv** se observa:


1.   El dataframe posee 567 filas y 6 columnas
1.   Las columnas poseen el tipo de dato correcto.
4.   No hay campos nulos

### 3️⃣ Calidad de los datos

In [ ]:
# Función para identificar los duplicados

def duplicados(df, nombre,clave=None):
   print(f"***** Duplicados en {nombre} *****")
   print("_" * 50)
   print()
   dup_rows = df.duplicated(keep=False).sum()
   print("Filas duplicadas (exactas):", dup_rows)
   if clave and clave in df.columns:
        dup_key = df[clave].duplicated(keep=False).sum()
        print(f"Duplicados por clave '{clave}':", dup_key)
        if dup_key > 0:
             duplicados_ordenados = (
                df[df[clave].duplicated(keep=False)][clave]
                .value_counts()                # Cuenta cuántas veces aparece cada valor
                .sort_values(ascending=False)   # Ordena de mayor a menor (más duplicados arriba)
              )
             print("\n🔁 Top valores duplicados más frecuentes:")
             # Mostrar solo los primeros 10 (los más repetidos)
             display(duplicados_ordenados.head(10))
        else:
            print(f"No se encontraron duplicados en la clave '{clave}'.")
   else:
        # Si la clave no fue pasada o no existe en el DataFrame
        if clave:
            print(f"La clave '{clave}' no existe en el DataFrame.")
        else:
            print("No se indicó una clave para analizar duplicados por columna.")

In [ ]:
duplicados(df_ventas,"Ventas",clave="id_venta")

In [ ]:
duplicados(df_marketing,"Marketing",clave="id_campanha")


In [ ]:
duplicados(df_clientes,"Clientes",clave="id_cliente")

## 🧹 Etapa 2: Preprocesamiento y Limpieza de Datos
**Objetivo:** Demostrar conocimiento de las técnicas de limpieza y transformación de datos.

### 4️⃣ Limpieza de datos

In [ ]:
# Limpio el conjunto de datos eliminando duplicados y caracteres no deseados.

# Primero creo copias independientes para no modificar los df originales

df_ventas_clean = df_ventas.copy()
df_marketing_clean = df_marketing.copy()
df_clientes_clean = df_clientes.copy()

In [ ]:
# Elimino filas duplicadas completas en cada df

df_ventas_clean = df_ventas_clean.drop_duplicates()
df_marketing_clean = df_marketing_clean.drop_duplicates()
df_clientes_clean = df_clientes_clean.drop_duplicates()



In [ ]:
# Verifico que ya no existan duplicados en los df_clean

print()
duplicados(df_ventas_clean,"Ventas",clave="id_venta")
print("Shape:  ", df_ventas_clean.shape)
print("_" * 50)
print()
duplicados(df_marketing_clean,"Marketing",clave="id_campanha")
print("Shape:  ", df_marketing_clean.shape)
print("_" * 50)
print()
duplicados(df_clientes_clean,"Clientes",clave="id_cliente")
print("Shape:  ", df_clientes_clean.shape)
print("_" * 50)

In [ ]:
# Normalizo fechas

def fecha_norm (df):
  flag=True
  for col in df.columns:
        if "fecha" in col.lower():  # detecta columnas con la palabra "fecha"
            df[col] = pd.to_datetime(df[col], errors="coerce", dayfirst=True)
            print(f"Columna '{col}' normalizada a '{df[col].dtypes}'")
            flag=False
            print("-" * 50)

  if flag:
      print("No se encontraron columnas con el patrón 'fecha' en el DataFrame.")


In [ ]:
fecha_norm(df_ventas_clean)


In [ ]:
fecha_norm(df_marketing_clean)

In [ ]:
fecha_norm(df_clientes_clean)

In [ ]:
# Función para normalizar texto

def normalizar_texto(df):
    for col in df.select_dtypes(include="object").columns:
        df[col] = (                                 # Usar () cuando va en distintas lineas
            df[col]
            .astype(str)                              # Convierte cualquier tipo a string
            .str.strip()                               # Elimina espacios al inicio y final
            .str.replace(r"[\u200b\t\r\n]", "", regex=True) # Borra tabs, saltos, etc
            .str.replace(" +", " ", regex=True)             # Reemplaza espacios consecutivos x 1
            .str.title()                               # Convierte a Título
        )

        #df[col] = df[col].astype(str).str.strip().str.replace(r"[\u200b\t\r\n]", "", regex=True).str.replace(" +", " ", regex=True).str.title()
    return df


In [ ]:
df_ventas_clean = normalizar_texto(df_ventas_clean)
print("-" * 100)
print("Texto normalizado en df_ventas_clean")
df_marketing_clean = normalizar_texto(df_marketing_clean)
print("-" * 100)
print("Texto normalizado en df_marketing_clean")
df_clientes_clean = normalizar_texto(df_clientes_clean)
print("-" * 100)
print("Texto normalizado en df_clientes_clean")

In [ ]:
# Normalizo campos numéricos

# Columnas de PRECIO

def normalizar_precio(df):
  for col in df.columns:
    flag=True
    if "precio" in col.lower():
      df[col]=df[col].astype(str)
      df[col]=df[col].str.replace("$","",regex=False)
      df[col]=df[col].str.replace(",","",regex=False)
      df[col]=df[col].str.strip()
      df[col]=pd.to_numeric(df[col],errors="coerce")
      print("Precio normalizado en df_ventas_clean")
      flag=False
      print("-" * 50)

  if flag:
      print("No se encontraron columnas con el patrón 'precio' en el DataFrame.")
      print("-" * 50)

  return df



  # CANTIDAD / Cambia de flotante a entero

def normalizar_flot_a_int(df,col):
  #df[col]=df[col].astype(str)
  #df[col]=df[col].str.replace(".0","",regex=False)
  df[col]=pd.to_numeric(df[col],errors="coerce").astype("Int64")
  return df
#
  #if "cantidad" in ventas_clean.columns:
    #ventas_clean["cantidad"] = pd.to_numeric(
     #   ventas_clean["cantidad"], errors="coerce"
    #).astype("Int64")
    # .astype("Int64") usa el tipo entero de pandas que permite valores nulos (NaN)

In [ ]:
df_ventas_clean = normalizar_precio(df_ventas_clean)
df_ventas_clean = normalizar_flot_a_int(df_ventas_clean,"cantidad")
print("-" * 100)


In [ ]:
# Verifico que los tipos de datos esten correctos en cada dataframe.

print("\n****** Archivo ventas.csv ******")
print("Shape :",df_ventas_clean.shape)
print("Dtypes:")
print(df_ventas_clean.dtypes)
display(df_ventas_clean.head())
print()
print("_"*80)
print("\n****** Archivo marketing.csv ******")
print("Shape :",df_marketing_clean.shape)
print("Dtypes:")
print(df_marketing_clean.dtypes)
display(df_marketing_clean.head())
print()
print("_"*80)
print("\n****** Archivo clientes.csv ******")
print("Shape :",df_clientes_clean.shape)
print("Dtypes:")
print(df_clientes_clean.dtypes)
display(df_clientes_clean.head())
print("_"*80)

In [ ]:
# Guardo los DF en archivos .csv

df_ventas_clean.info()
df_ventas_clean.to_csv("/content/drive/MyDrive/datasets/ventas_clean.csv", index=False)
df_clientes_clean.to_csv("/content/drive/MyDrive/datasets/clientes_clean.csv", index=False)
df_marketing_clean.to_csv("/content/drive/MyDrive/datasets/marketing_clean.csv", index=False)

print("✅ Archivos guardados: ventas_clean.csv, clientes_clean.csv, marketing_clean.csv")

### 5️⃣ Transformación de datos

In [ ]:
# Crear una tabla de ventas que muestre solo los productos con alto rendimiento.
# Calcular el percentil 80
# Filtrar los productos que superen ese umbral en ventas.

# Calculo el ingreso y creo un nuevo df con la columna "ingreso"
df_ventas_ingresos = df_ventas_clean.assign(ingreso = df_ventas_clean["precio"] * df_ventas_clean["cantidad"])

# Agrupo filas y obtengo métricas
resumen_prod = (
    df_ventas_ingresos.groupby(by="producto",dropna=False,as_index=False,observed=False)
    .agg(
        ingreso_total=('ingreso', 'sum'),
        unidades=('cantidad', 'sum'),
        precio_promedio=('precio', 'mean'),
        registros=('ingreso', 'size'))
)

# Redondeo el 'precio_promedio'
resumen_prod['precio_promedio'] = resumen_prod['precio_promedio'].round(2)

#obtengo percentil 80
p80_ingreso=resumen_prod["ingreso_total"].quantile(q=0.8,interpolation="linear")
#print("Percentil 80: ",p80_ingreso.round(2))

# Filtro los productos de alto rendimiento y ordeno de mayor a menor
prod_alto_rendimiento = (
    resumen_prod.query("ingreso_total >= @p80_ingreso",engine="python")
    .sort_values(by=["ingreso_total","unidades"],ascending=[False,False],ignore_index=True)
)

# Muestro resultados
print()
print("**  Columna: Producto  **")
print(f"Umbral (percentil 80) de ingreso_total: {float(p80_ingreso):,.2f}")
print("✅ Productos de ALTO RENDIMIENTO (top 20% por ingreso):")
display(prod_alto_rendimiento.head(20))

### 6️⃣ Agregación

In [ ]:
# TODO: Resumir las ventas por categoría de producto y analizar los ingresos generados.
# Sugerencia: usar .groupby() y .agg() para generar métricas como suma y promedio.

# Calculo el ingreso y creo un nuevo df con la columna "ingreso"
df_ventas_cat = df_ventas_clean.assign(ingreso = df_ventas_clean["precio"] * df_ventas_clean["cantidad"])

# Agrupo filas y obtengo métricas
resumen_cat = (
    df_ventas_cat.groupby(by="categoria",dropna=False,as_index=False)
    .agg(
        ingreso_total=('ingreso', 'sum'),
        unidades=('cantidad', 'sum'),
        precio_promedio=('precio', 'mean'),
        registros=('ingreso', 'size')
        )
    .sort_values(
        by='ingreso_total',
        ascending=False,
        na_position='last',
        ignore_index=True
    )
)

# Redondeo el 'precio_promedio'
resumen_cat['precio_promedio'] = resumen_cat['precio_promedio'].round(2)


# Muestro resultados
print()
print("**  Columna: Categoría  **")
print("Resumen por categoría (ordenado por ingreso_total):")
display(resumen_cat.head(20))